<a href="https://colab.research.google.com/github/OCanSagbas/Image-Classification-Using-Cross-Correlation/blob/main/Image_Classification_Using_Cross_Correlation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
from skimage.feature import hog
from google.colab import drive
from google.colab.patches import cv2_imshow
import os
import string


In [ ]:
drive.mount('/content/drive')
target_path = 'drive/MyDrive/dataset/target'
template_path = 'drive/MyDrive/dataset/template'

In [ ]:
# NCC fonksiyonu
def normalized_cross_correlation(template, target):
    result = cv2.matchTemplate(target, template, cv2.TM_CCORR_NORMED)
    _, max_val, _, max_loc = cv2.minMaxLoc(result)
    return max_val, max_loc

# HOG ve Grayscale uygulayan fonksiyon
def preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    hog = cv2.HOGDescriptor()
    features = hog.compute(img)
    return img, features

# Template'leri yükle
templates = []
template_names = []
for template_name in os.listdir(template_path):
    template_image_path = os.path.join(template_path, template_name)
    template, _ = preprocess_image(template_image_path)
    templates.append(template)
    template_names.append(template_name)

# Her bir template için en yüksek korelasyon değeri, bu değere karşılık gelen target adını ve eşleşen bölgeyi tutacak sözlük
max_correlations = {template_name: {'max_correlation': 0, 'max_target_name': '', 'max_loc': (0, 0)} for template_name in template_names}

# Target'ları ve her bir target içindeki görselleri yükle
for target_folder_name in os.listdir(target_path):
    target_folder_path = os.path.join(target_path, target_folder_name)

    for template_name, template in zip(template_names, templates):
        max_correlation, max_loc = 0, (0, 0)
        max_target_name = ""

        for target_image_name in os.listdir(target_folder_path):
            target_image_path = os.path.join(target_folder_path, target_image_name)
            target, target_features = preprocess_image(target_image_path)

            correlation, loc = normalized_cross_correlation(template, target)

            if correlation > max_correlation:
                max_correlation = correlation
                max_target_name = target_image_name
                max_loc = loc

        # Sözlüğü güncelle
        if max_correlation > max_correlations[template_name]['max_correlation']:
            max_correlations[template_name]['max_correlation'] = max_correlation
            max_correlations[template_name]['max_target_name'] = f"{target_folder_name}/{max_target_name}"
            max_correlations[template_name]['max_loc'] = max_loc

# Sonuçları yazdır ve görselleri çiz
for template_name, values in max_correlations.items():
    print(f"Template: {template_name}, Max Correlation: {values['max_correlation']}, Max Target: {values['max_target_name']}")

    # Eğer max_target_name değeri boşsa hata verme ve devam et
    if not values['max_target_name']:
        print("No matching target image.")
        continue

    # Eşleşen görseli ve dikdörtgen sınırlayıcı kutuyu al
    target_folder, target_name = values['max_target_name'].split('/')
    target_image_path = os.path.join(target_path, target_folder, target_name)
    template_image_path = os.path.join(template_path, template_name)

    target_img = cv2.imread(target_image_path)
    template_img = cv2.imread(template_image_path)

    # Eşleşen bölgeyi bul
    h, w = template_img.shape[:2]
    top_left = values['max_loc']
    bottom_right = (top_left[0] + w, top_left[1] + h)

    # Dikdörtgen çiz
    cv2.rectangle(target_img, top_left, bottom_right, (255, 0, 0), 2)

    # Görselleri göster
    cv2_imshow(template_img)
    cv2_imshow(target_img)
